# League Table Calculator

In this module we will populate the LeagueTable database table.
For each league and season, in this table we will store the league table at the end of each stage of the league.

The league table will be populated with teams data such as: total points at home, totat points at away, average points
at home, etc.

In [1]:
import nbimporter
import db_utils

Importing Jupyter notebook from db_utils.ipynb


In [2]:
"""
This function takes in input the number of goals for home team, for away team and a flag 'for_home' that indicates for 
which team we want to calcuate the number of points taken.
If 'for_home' is True, we want to have the points taken by the home team given the number of goals scored by home and away
team.
"""
def points(homeG,awayG,for_home):
    if ( homeG == awayG ): return 1
    if( for_home == True ):
        if( homeG > awayG ): return 3
        return 0
    if( homeG < awayG ): return 3
    return 0

In [3]:
"""
Next, we will do the dirty job.
For each league and season, firstly we will obtain the number of stages that the league is composed of.
For every stage from 1 to the overall number of stages:
- if the stage == 1: we will populate the LeagueTable table with default 0-values, because before the first stage
                     we have no data about the teams
- if the stage != 1: for both the home and away teams, we get the team data for the previous stage, we add to this data
                     the data for the current stage, and insert into the LeagueTable table the new data.
"""

'\nNext, we will do the dirty job.\nFor each league and season, firstly we will obtain the number of stages that the league is composed of.\nFor every stage from 1 to the overall number of stages:\n- if the stage == 1: we will populate the LeagueTable table with default 0-values, because before the first stage\n                     we have no data about the teams\n- if the stage != 1: for both the home and away teams, we get the team data for the previous stage, we add to this data\n                     the data for the current stage, and insert into the LeagueTable table the new data.\n'

In [14]:
db_file = 'database.sqlite'

leagues = ['1','1729','4769','7809','10257','13274','15722','17642','19694','21518','24558']
seasons = ['2008/2009','2009/2010','2010/2011','2011/2012','2012/2013','2013/2014','2014/2015','2015/2016']

for league in leagues:
    for season in seasons:
        num_stages = 0
        rs1 = query(db_file, "select count(distinct stage) from Match where league_id = ? and season = ?",(league,season))
        for row in rs1:
            num_stages = row[0]
            break
        if ( num_stages == 0 ): continue

        for stage in range(1, num_stages+1):
            matches_for_stage = []
            rs2 = query(db_file,"select id, home_team_api_id, away_team_api_id from match where season = ? and league_id = ? and stage = ?",(season,league,stage))
            for match_data in rs2:
                match_id = match_data[0]
                home = match_data[1]
                away = match_data[2]
                rs3 = query(db_file,"select * from MatchEvent where match_id = ?",(match_id,))
                for match_event in rs3:
                    homeG = match_event[1]
                    awayG = match_event[2]
                    home_shoton = match_event[3]
                    away_shoton = match_event[4]
                    home_shotoff = match_event[5]
                    away_shotoff = match_event[6]
                    home_yellow = match_event[7]
                    away_yellow = match_event[8]
                    home_red = match_event[9]
                    away_red = match_event[10]
                    home_foul = match_event[11]
                    away_foul = match_event[12]
                    home_corner = match_event[13]
                    away_corner = match_event[14]
                    home_cross = match_event[15]
                    away_cross = match_event[16]
                    home_poss = match_event[17]
                    away_poss = match_event[18]
                    
                    home_points = points(homeG, awayG, True)
                    away_points = points(homeG, awayG, False)
                    
                    sql = "INSERT INTO LEAGUETABLE VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"
                    if ( stage == 1 ):
                        write(db_file, sql, (league,season,stage,home,1,0,home_points,0,home_points,0,homeG,0,homeG,0,awayG,0,awayG,0,
                                    home_shoton,0,home_shotoff,0,home_yellow,0,home_red,0,home_foul,0,home_cross,0,home_corner,0,home_poss,0))

                        write(db_file, sql, (league,season,stage,away,0,1,0,away_points,0,away_points,0,awayG,0,awayG,0,homeG,0,homeG,
                                    0,away_shoton,0,away_shotoff,0,away_yellow,0,away_red,0,away_foul,0,away_cross,0,away_corner,0,away_poss))
                        
                    else:
                        # take home data
                        rs4 = query(db_file, "select * from LeagueTable where league_id = ? and season = ? and stage = ? and team_id = ?", (league,season,stage-1,home))
                        for rec_rs4 in rs4:
                            rec_played_aH = rec_rs4[4]
                            rec_points_tot_aH = rec_rs4[6]
                            rec_points_avg_aH = rec_rs4[8]
                            rec_scored_tot_aH = rec_rs4[10]
                            rec_scored_avg_aH = rec_rs4[12]
                            rec_conceived_tot_aH = rec_rs4[14]
                            rec_conceived_avg_aH = rec_rs4[16]
                            rec_home_shoton_aH = rec_rs4[18]
                            rec_home_shotoff_aH = rec_rs4[20]
                            rec_home_yellow_aH = rec_rs4[22]
                            rec_home_red_aH = rec_rs4[24]
                            rec_home_foul_aH = rec_rs4[26]
                            rec_home_cross_aH = rec_rs4[28]
                            rec_home_corner_aH = rec_rs4[30]
                            rec_home_poss_aH = rec_rs4[32]

                            new_played_aH = rec_played_aH + 1
                            new_points_tot_aH = rec_points_tot_aH + home_points
                            new_points_avg_aH = round(float( (rec_points_avg_aH*(new_played_aH-1) + home_points) / (new_played_aH) ),2)
                            new_scored_tot_aH = rec_scored_tot_aH + homeG
                            new_scored_avg_aH = round(float( (rec_scored_avg_aH*(new_played_aH-1) + homeG) / (new_played_aH) ),2)
                            new_conceived_tot_aH = rec_conceived_tot_aH + awayG
                            new_conceived_avg_aH = round(float( (rec_conceived_avg_aH*(new_played_aH-1) + awayG) / (new_played_aH) ),2)
                            
                            new_home_shoton = rec_home_shoton_aH + home_shoton
                            new_home_shotoff = rec_home_shotoff_aH + home_shotoff
                            new_home_yellow = rec_home_yellow_aH + home_yellow
                            new_home_red = rec_home_red_aH + home_red
                            new_home_foul = rec_home_foul_aH + home_foul
                            new_home_cross = rec_home_cross_aH + home_cross
                            new_home_corner = rec_home_corner_aH + home_corner
                            new_home_poss = int( (rec_home_poss_aH*(new_played_aH-1) + home_poss) / (new_played_aH) )
      
                            write(db_file, sql, 
                                  (league,season,stage,home,new_played_aH,rec_rs4[5],new_points_tot_aH,rec_rs4[7],
                                   new_points_avg_aH,rec_rs4[9],new_scored_tot_aH,rec_rs4[11],new_scored_avg_aH,
                                   rec_rs4[13],new_conceived_tot_aH,rec_rs4[15],new_conceived_avg_aH,rec_rs4[17],
                                   new_home_shoton,rec_rs4[19],new_home_shotoff,rec_rs4[21],new_home_yellow,rec_rs4[23],
                                   new_home_red,rec_rs4[25],new_home_foul,rec_rs4[27],new_home_cross,rec_rs4[29], 
                                   new_home_corner,rec_rs4[31], new_home_poss, rec_rs4[33]
                                   )
                            )
                            break;

                        # take away data
                        rs5 = query(db_file, "select * from LeagueTable where league_id = ? and season = ? and stage = ? and team_id = ?", (league,season,stage-1,away))
                        for rec_res5 in rs5:
                            rec_played_aA = rec_res5[5]
                            rec_points_tot_aA = rec_res5[7]
                            rec_points_avg_aA = rec_res5[9]
                            rec_scored_tot_aA = rec_res5[11]
                            rec_scored_avg_aA = rec_res5[13]
                            rec_conceived_tot_aA = rec_res5[15]
                            rec_conceived_avg_aA = rec_res5[17]
                            
                            rec_away_shoton_aA = rec_res5[19]
                            rec_away_shotoff_aA = rec_res5[21]
                            rec_away_yellow_aA = rec_res5[23]
                            rec_away_red_aA = rec_res5[25]
                            rec_away_foul_aA = rec_res5[27]
                            rec_away_cross_aA = rec_res5[29]
                            rec_away_corner_aA = rec_res5[31]
                            rec_away_poss_aA = rec_res5[33]
                            
                            new_played_aA = rec_played_aA + 1
                            new_points_tot_aA = rec_points_tot_aA + away_points
                            new_points_avg_aA = float( (rec_points_avg_aA*(new_played_aA-1) + away_points) / (new_played_aA) ),2)
                            new_scored_tot_aA = rec_scored_tot_aA + awayG
                            new_scored_avg_aA = float( (rec_scored_avg_aA*(new_played_aA-1) + awayG) / (new_played_aA) ),2)
                            new_conceived_tot_aA = rec_conceived_tot_aA + homeG
                            new_conceived_avg_aA = float( (rec_conceived_avg_aA*(new_played_aA-1) + homeG) / (new_played_aA) ),2)
                            
                            new_away_shoton = rec_away_shoton_aA + away_shoton
                            new_away_shotoff = rec_away_shotoff_aA + away_shotoff
                            new_away_yellow = rec_away_yellow_aA + away_yellow
                            new_away_red = rec_away_red_aA + away_red
                            new_away_foul = rec_away_foul_aA + away_foul
                            new_away_cross = rec_away_cross_aA + away_cross
                            new_away_corner = rec_away_corner_aA + away_corner
                            new_away_poss = int( (rec_away_poss_aA*(new_played_aA-1) + away_poss) / (new_played_aA) )

                            write(db_file, sql, 
                                    (league,season,stage,away,rec_res5[4],new_played_aA,rec_res5[6],new_points_tot_aA,
                                     rec_res5[8],new_points_avg_aA,rec_res5[10],new_scored_tot_aA,rec_res5[12],
                                     new_scored_avg_aA,rec_res5[14],new_conceived_tot_aA,rec_res5[16],new_conceived_avg_aA,
                                     rec_res5[18],new_away_shoton,rec_res5[20],new_away_shotoff,rec_res5[22],new_away_yellow,
                                     rec_res5[24],new_away_red,rec_res5[26],new_away_foul,rec_res5[28],new_away_cross, 
                                     rec_res5[30],new_away_corner,rec_res5[32],new_away_poss 
                                    )
                            )
                            break;

BEGIN LEAGUE 1 and SEASON 2008/2009
END LEAGUE 1 and SEASON 2008/2009
BEGIN LEAGUE 1 and SEASON 2009/2010
END LEAGUE 1 and SEASON 2009/2010
BEGIN LEAGUE 1 and SEASON 2010/2011
END LEAGUE 1 and SEASON 2010/2011
BEGIN LEAGUE 1 and SEASON 2011/2012
END LEAGUE 1 and SEASON 2011/2012
BEGIN LEAGUE 1 and SEASON 2012/2013
END LEAGUE 1 and SEASON 2012/2013
BEGIN LEAGUE 1 and SEASON 2013/2014
BEGIN LEAGUE 1 and SEASON 2014/2015
END LEAGUE 1 and SEASON 2014/2015
BEGIN LEAGUE 1 and SEASON 2015/2016
END LEAGUE 1 and SEASON 2015/2016
BEGIN LEAGUE 1729 and SEASON 2008/2009
END LEAGUE 1729 and SEASON 2008/2009
BEGIN LEAGUE 1729 and SEASON 2009/2010
END LEAGUE 1729 and SEASON 2009/2010
BEGIN LEAGUE 1729 and SEASON 2010/2011
END LEAGUE 1729 and SEASON 2010/2011
BEGIN LEAGUE 1729 and SEASON 2011/2012
END LEAGUE 1729 and SEASON 2011/2012
BEGIN LEAGUE 1729 and SEASON 2012/2013
END LEAGUE 1729 and SEASON 2012/2013
BEGIN LEAGUE 1729 and SEASON 2013/2014
END LEAGUE 1729 and SEASON 2013/2014
BEGIN LEAGUE 1729 